# Setup the OpenVINO Lab Environment

This notebook tests for the existence of all the files and directories necessary to run the OpenVINO Jupyter Lab Notebooks, and configures environmental variables to provide easy access to the various directory locations that will be use in this workshop. The helps the student focus on the learning the material rather than tracking the all directories referenced.

* First, we search for all OpenVINO directories and test that they exist
* Then we setup the environmental variables to hold the directory paths that will be used during this workshop.


## Configuration Section

** *This is the only section of this notebook that should be modified. The notebook will automatically search for and setup all paths in use during this lab. However, you can override and set custom paths here.* **

In [10]:
# Customize the defaults for this workshop
workshop_directory = "/home/upsquared/labs/smart-video-workshop" 

# The current user will be set below, unless you set an alternative username here.
user = None

# Password used for sudo user
password = "upsquared"

## Python Section
### Import Modules used in this Workshop

In [4]:
from colors import *
import os, sys, getpass, time
from urllib import request

## Define Helper Functions

In [5]:
# Test for the existance of a file or directory
# Returns the path if it exists and halt the notebook if it does not exist
def verify(path):
    if not os.path.exists(path):
        print(color('Unable to find directory {0}'.format(path), fg='red', style='bold')) 
        assert False
    else:
        return os.path.abspath(path)

    
# Creates a directory if it does not already exist    
def create(path):
    if not os.path.exists(path):
        os.makedirs(path)

## Get the Current User

When OpenVINO is installed in the user's home dirctory, we need to know the name of the current user to find the OpenVINO installation path

In [6]:
# You can manually set the name of the current user on the line below
user = None

# If the variable 'user' is None then find the current user
user = user or getpass.getuser()

## Test the Existence of the OpenVINO Installation Paths
Find the OpenVINO installation path by first checking the standard system installation directory. If it is not found in */opt/intel/* then check the home directory of the user.

In [7]:
# When OpenVINO is installed by root or via the 'sudo' command, it will
# be in the '/opt/intel' directory
openvino_global_install_directory = "/opt/intel/computer_vision_sdk/"

# When OpenVINO is installed by a user, it will be in user's home 
# directory under './intel'
openvino_user_install_directory = os.path.join("/home",  user, "intel/computer_vision_sdk/")

# Set 'openvino_install_path' or display an error
if os.path.isdir(openvino_global_install_directory):
    openvino_install_path = verify(openvino_global_install_directory)
elif os.path.isdir(openvino_user_install_directory):
    openvino_install_path = verify(openvino_user_install_directory)
else:
    print(color('Unable to find OpenVINO installation directory', fg='red', style='bold'))

### OpenVINO Directory Definitions

In [8]:
openvino_inference_engine_examples = verify(os.path.join(openvino_install_path, "deployment_tools/inference_engine/samples/"))
openvino_model_optimizer = verify(os.path.join(openvino_install_path, "deployment_tools/model_optimizer"))
openvino_model_downloader = verify(os.path.join(openvino_install_path, "deployment_tools/model_downloader"))
openvino_examples_bin = verify(os.path.join(openvino_install_path, "deployment_tools/inference_engine/samples/build/intel64/Release"))

## Define All Environmental Variables
### Set the Sudo Password 

In [9]:
print(color('Initializing operating system environment variables', fg='blue', style='bold'))
%env PASSWORD = $password

Initializing operating system environment variables
env: PASSWORD=$password


### Set OpenVINO directories

The initial step is to determine if the OpenVINO libraries were installed globally or if they are installed in the home directory of a particular user.

In [10]:
print(color('\nStoring OpenVINO directories in environment variables', fg='blue', style='bold'))
%env OPENVINO_INSTALLATION_PATH = $openvino_install_path
%env OPENVINO_EXAMPLES = $openvino_inference_engine_examples
%env OPENIVNO_MODEL_OPTIMIZER = $openvino_model_optimizer
%env OPENVINO_MODEL_DOWNLOADER = $openvino_model_downloader
%env OPENVINO_EXAMPLES_BIN = $openvino_examples_bin


Storing OpenVINO directories in environment variables
env: OPENVINO_INSTALLATION_PATH=/home/upsquared/intel/computer_vision_sdk
env: OPENVINO_EXAMPLES=/home/upsquared/intel/computer_vision_sdk/deployment_tools/inference_engine/samples
env: OPENIVNO_MODEL_OPTIMIZER=/home/upsquared/intel/computer_vision_sdk/deployment_tools/model_optimizer
env: OPENVINO_MODEL_DOWNLOADER=/home/upsquared/intel/computer_vision_sdk/deployment_tools/model_downloader
env: OPENVINO_EXAMPLES_BIN=/home/upsquared/intel/computer_vision_sdk/deployment_tools/inference_engine/samples/build/intel64/Release


### Set Workshop Labs Directory

* The first step is to set the workshop directory into an environment variable called WORKSHOP
* Create a directory for the model_optimizer to write the Intermediate Representation Models
* Create a directory for the image and video inputs


In [11]:
print(color('\nStoring workshop lab directories in environment variables', fg='blue', style='bold'))
workshop = "/home/upsquared/labs/smart-video-workshop"
%env WORKSHOP = $workshop

# Create the output directory for models generated by the model optimizer
model_output_dir = os.path.join(workshop, "resources/model_output")
create(model_output_dir)
%env MODEL_OUTPUT_DIR = $model_output_dir

# Create the directory to hold input images and videos
input_media_dir = os.path.join(workshop, "resources/media")
create(input_media_dir)
%env INPUT_MEDIA_DIR = $input_media_dir


Storing workshop lab directories in environment variables
env: WORKSHOP=/home/upsquared/labs/smart-video-workshop
env: MODEL_OUTPUT_DIR=/home/upsquared/labs/smart-video-workshop/resources/model_output
env: INPUT_MEDIA_DIR=/home/upsquared/labs/smart-video-workshop/resources/media


### Verify the Neural Network Models are Present

Check for the existence of the **classification** and **object_detction** model directories. 
If the models are not present then start the **model_downloader.py** script.

In [24]:
! if [ -f $OPENVINO_MODEL_DOWNLOADER/classification ] && [ -f $OPENVINO_MODEL_DOWNLOADER/object_detection ]; then python3 $OPENVINO_MODEL_DOWNLOADER/downloader.py --output_dir $OPENVINO_MODEL_DOWNLOADER; fi 

### Verify that the Video and Image Assets are Avialable

In [21]:
# Create a file download progress bar        
def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = int(progress_size / (1024 * duration))
    percent = int(count * block_size * 100 / total_size)
    sys.stdout.write("\r ...%d%%, %d MB, %d KB/s, %d seconds passed" % (percent, progress_size / (1024 * 1024), speed, duration))
    
    
# A function that downloads a file from a remote URL and saves it locally
def save(url, filename):
    if not os.path.exists(filename):
        request.urlretrieve(url, filename, reporthook)

In [22]:
save("https://pixabay.com/en/videos/download/video-1900_source.mp4", os.path.join(input_media_dir, "cars.mp4"))

### Display Video Inline

The first helper function will display a video in Jupyter Labs from the device that Jupyter is running


In [16]:
def show(fname=None, width=480, height=320):
    """Function to display any video in Ipython or Jupyter Notebook given a directory in which the video exist and the video file name.
    Args:
        baseDir: Directory containing the video
        fname: Filename of video.
    """
    from IPython.display import HTML
    import os
    location = fname
    location = "../object-detection/cars.mp4"
    if os.path.isfile(location):
        ext = '.mp4'
    else:
        print("Error: Please check the path.")

    video_tag = '<video width="{0}" height="{1}" controls alt="test" src="{2}">'.format(width, height, location)
    return HTML(data=video_tag)

In [18]:
show(os.path.join(input_media_dir, "cars.mp4"))

Error: Please check the path.
